<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>Автор материала: Артём Асмоловский (@Asmolovskij)

Приветствую! Перед вами решение задачи по определению дефолта у ипотечных заёмщиков США, содержащей более 5950 наблюдений. Безусловно, такая задача интересна и сама по себе. Обучив модель, можно предсказывать дальнейшее развитие событий относительно каждого конкретно взятого заёмщика. Однако, скорее всего, подобные данные не останются лишь в стенах банков, а используются различными организациями, имеющими дела с социумом. Вполне вероятно, что по данному набору данных можно попытаться извлечь некий социальный портрет каждого объекта, а затем пытаться кластеризовать их, преследуя уже совсем другие цели. например, предпочтения относительно дорогих покупок, если задачу решает некий ритейлер, либо добавочный коэффициент страхования, если заинтересованы страховщики. Забавная ситуация произошла в офисе одного из амеркианских магазинов Target, предсказав у одной из своих посетительниц беременность, отнеся её к кластеру точно беременных клиенток на основе их поведения. 

Однако я всё же буду решать задачу детектирования наступления дефолта у заёмщика, либо его отсутствия. Анализируемый датасет имеет следующие переменные. Итак, предикторы:

LOAN - сумма запроса кредита

MORTDUE - текущая сумма ипотеки

VALUE - стоимость текущей недвижимости

REASON: DebtCon - погашение задолженности, HomeImp - улучшение жилищных условий

JOB - род занятости

YOJ - количество лет на текщей работе

DEROG - количество негативных пометок на кредитной истории

DELINQ - количество просроченных кредитных погашений

CLAGE - количество в месяцах самой старой кредитной истории

NINQ - количество последних кредитных запросов

CLNO - количество кредитов

DEBTINC - отношение долгов к доходам

А предсказывать мы будем дефолт

BAD: 1 - не погасит, 0 - погасит

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Подгрузим данные
df = pd.read_csv('../../data/hmeq.csv')
df.head(10)

In [ ]:
#Посмотрим на общую характеристику данных
df.describe()

In [ ]:
df.info()

Как видно, в данных имеется немало пропусков. и если такие переменные как VALUE или CLAGE имеют относительное небольшое число пробелов, которые вполне безболезненно можно заменить на средние или медианные значения по столбцу, то с переменной DEBTINC дела обстоят несоклько сложнее в первую очередь из-за большого количества пропусков (порядка 20%). Однако взглянув на описательную таблицу выше, можно заметить, что среднее практически совпадает с медианой. Что ж, в таком случае так же будем использовать cреднее или медианное для восполнения пробелов.

In [ ]:
#Заполним пропуски для того, чтобы приступить к удобному визуальному анализу.

#В большинстве случаев будем порльзоваться медианным значением
df[['MORTDUE']] = df[['MORTDUE']].fillna(65000)
df[['VALUE']] = df[['VALUE']].fillna(89200)
df[['YOJ']] = df[['YOJ']].fillna(7)
df[['MORTDUE']] = df[['MORTDUE']].fillna(65000)
df[['DEROG']] = df[['DEROG']].fillna(0)
df[['DEROG']] = df[['DEROG']].fillna(0)
df[['DELINQ']] = df[['DELINQ']].fillna(0)
df[['CLAGE']] = df[['CLAGE']].fillna(179)
df[['NINQ']] = df[['NINQ']].fillna(0)
df[['CLNO']] = df[['CLNO']].fillna(0)
df[['DEBTINC']] = df[['DEBTINC']].fillna(0)

In [ ]:
#Бинаризуем переменную Reason
df['REASON'] = df['REASON'].map({'DebtCon' : 0, 'HomeImp' : 1})

In [ ]:
df['REASON'].value_counts()
#Как видим, больше половины объектов относятся к классу 0, поэтому заполним пропуски так же на 0

In [ ]:
df['REASON'] = df['REASON'].fillna(0)

In [ ]:
#Категоризуем переменную JOB
encoder = LabelEncoder().fit_transform(df["JOB"].astype(str))
df[["JOB"]] = encoder

Посомтрим на анализируемый датасет теперь и посмотрим, удалось ли нам полностью подготовить данные для дальнейшего анализа.

In [ ]:
df.head(10)

In [ ]:
df.info()

Пропусков действительно нет, приступим к графическому анализу.

In [ ]:
sns.pairplot(df[['BAD', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'NINQ', 'CLNO', 'DEBTINC']])

Каких-то явных закономерностей, помиомо того, что с ростом стоимости приобритаемого жилья растёт и стоимость ипотеки, нет. Стоит отметить почти идентичную зависимость у количества негативных пометок на кредитной истории и количества просроченных платежей относительно суммы займа, причём сумма займа в общей массе не превышает 0,25 квартиля, что свидетельствует об относительно небольших займах.

Интересно, а есть ли какие-нибудь профессии, среди которых присутствует наибольшее количество злостных неплательщиков, либо кредитных потребителей?

In [ ]:
sns.pairplot(df[['JOB', "DELINQ", "DEROG", 'NINQ']])

Нет, каких-то явных суждений на этот счёт сделать нельзя. На всякий случай дополнительно отрисуем корреляционную матрицу.

In [ ]:
sns.heatmap(df.corr())

А тут уже вырисовывется более интересная картинка. Предположение о наибольшей зависимости между стоимостью жилья и ипотеки выполняется. Но по началу я даже не обратил внимания, что между суммой ипотеки и заявкой на кредит так же существует зависимость. Более того, между длиной кредитной истории и вероятностью наступления дефолта существует заметная связь, но с отрицательным знаком. И сильная отрицательная (если даже не отрицательная функциональная) между отношением долгов к доходам и вероятностью дефолта.

Предварительно сделаем следующий вывод: С увеличением кредитных просрочек и негативнх пометках на кредитной истории, вероятность дефолта увеличивается. Однако, чем дольше заёмщик берёт кредиты, и чем ниже отношение долгов к доходам, тем менее вероятность наступления дефолта.

Для того, чтобы выбрать метрику, на основании которой мы будем строить модели, посмотрим на целевой признак.

In [ ]:
plt.hist(df['BAD'])

Видим явную несбалансировать классов, поэтому о простом accuracy_score можно забыть. В данном случе стоит подумать о бизнес-составляющей нашей задачи. Что важнее: отклассифицировать нехороших плательщиков, пожертвовав хорошими? Или найти как можно больше нехороших неплательщиков? Что ж, будем рисовать roc auc, чтобы оценивать модель в целом.

Поробуем 4 агоритма: логистическкую регрессию, метод к-ближайших, а так же случайный лес и градиентный бустинг (куда же без них).

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split,\
validation_curve, learning_curve

И займёмся финальной подготовкой данных: применим стандартизацию (чтобы избежать сильного влияния каких-то отдельных предикторов для линейки), извлечём целевой признак, а так же поделим выборку на тренировочную, валидационную и тестовую.

In [ ]:
y = df[['BAD']]
X = df.drop('BAD', axis = 1)

In [ ]:
#Создадим дамми-переменные для JOB
dummies_job = pd.get_dummies(df['JOB'], prefix = 'JOB')
X = pd.concat([X, dummies_job], axis = 1)
X = X.drop('JOB', axis = 1)

Извлечём из анализируемого датасета отложенную часть выборки, а для тренировочной извлечём ещё и валидационную.

In [ ]:
idx_split = X.shape[0] * 0.8

In [ ]:
X_train = X.iloc[:int(idx_split), :]
y_train = y.iloc[:int(idx_split), :]
X_test = X.iloc[int(idx_split) : , :]
y_test = y.iloc[int(idx_split) :, :]

In [ ]:
X_train_part, X_train_valid, y_train_part, y_train_valid = train_test_split(X_train, y_train, test_size = 0.3, shuffle = True)

Оценим работу алгоритмов, настраивая гиперпараметры по сетке. Так же необходимо стандартизировать все не бинарные признаки. Чтобы провести стандартизацию корректно, при кросс-валидации необходимо обучать StandardScaler на каждом фолде.

In [ ]:
#Приступаем к построениею моделей
#Поскольку у нас есть и категориальные, и непрерывные признаки, то, скорее всего, деревья будут лучше справляться

cv = StratifiedKFold(random_state=17, n_splits=5)

log_params = {'log__C' : [0.01, 0.05, 0.1, 0.25, 0.5, 1], 'log__random_state' : [17]}  
pipeline = Pipeline([('scaler', StandardScaler()), ('log', LogisticRegression())])
grid = GridSearchCV(pipeline, log_params, cv = cv)
grid.fit(X_train_part, y_train_part)
print('log best result: %s' % grid.best_score_)
print('log best params: %s' % grid.best_params_)

In [ ]:
knn_params = {'knn__n_neighbors' : range(1, 20)}  
pipeline = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])
grid = GridSearchCV(pipeline, knn_params, cv = cv)
grid.fit(X_train_part, y_train_part)
print('knn best result: %s' % grid.best_score_)
print('knn best params: %s' % grid.best_params_)

In [ ]:
rf_params = {'rf__n_estimators' : [100, 250, 500, 750, 1000], 'rf__max_depth' : [1, 2, 3, 4, 5, 6, 7], 'rf__random_state' : [17]}  
pipeline = Pipeline([('scaler', StandardScaler()), ('rf', RandomForestClassifier())])
grid = GridSearchCV(pipeline, rf_params, cv = cv)
grid.fit(X_train_part, y_train_part)
print('rf best result: %s' % grid.best_score_)
print('rf best params: %s' % grid.best_params_)

In [ ]:
xgb_params = {'xgb__max_depth' : [1,2,3,4,5, 6, 7], 'xgb__n_estimators':[100, 250, 500, 750, 1000], 'xgb__random_state': [17]}  
pipeline = Pipeline([('scaler', StandardScaler()), ('xgb', XGBClassifier())])
grid = GridSearchCV(pipeline, xgb_params, cv = cv)
grid.fit(X_train_part, y_train_part)
print('xgb best result: %s' % grid.best_score_)
print('xgb best params: %s' % grid.best_params_)

На текущий момент видно, что градиентный бустинг (с настройкой только глубины и количества деревьев) отрабатывает лучше остальных моделей. При этом можно замтить, что и случайный лес очень неплохо старается с настройкой тех же гиперпараметров, причём не факт, что их оптимальные значения были найдены. Однако для экономии вычислительных ресурсов остановимся на xgb.

Обучим данную модель на всей X_train и подсчитаем roc auc. Полученное значение будем считать бейзлайном для дальнейшего улучшения моделей.

In [ ]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000, n_jobs=-1, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
xgb.fit(X_train_scaled, y_train)
print("Результат выбранной модели: %s" % roc_auc_score(y_test, xgb.predict_proba(X_test_scaled)[:,1]))

Итак, с моделью мы определились. Самое время подумать о feature engineering. Воспользуемся здравым смыслом и посмотрим, как отличаются наиболее интересные переменные, вроде суммы займа, количества кредитов и прочих у способных к выплате и неспособных. (Предположение о наличии неплательщиков среди определённой профессии мы отклонили ранее).

In [ ]:
np.mean(df[df['BAD'] == 1]['LOAN']), np.mean(df[df['BAD'] == 0]['LOAN'])
#Величина кредитного запроса у неплательщиков ниже почти на 2000 долларов

In [ ]:
np.mean(df[df['BAD'] == 1]['MORTDUE']), np.mean(df[df['BAD'] == 0]['MORTDUE'])
#Так же ниже стоимость ипотеки

In [ ]:
np.mean(df[df['BAD'] == 1]['VALUE']), np.mean(df[df['BAD'] == 0]['VALUE'])
#И недвижимость дешевле

In [ ]:
np.mean(df[df['BAD'] == 1]['YOJ']), np.mean(df[df['BAD'] == 0]['YOJ'])
#Работают на основной работе как правило на полтора года меньше

In [ ]:
np.mean(df[df['BAD'] == 1]['DEROG']), np.mean(df[df['BAD'] == 0]['DEROG'])
#Большее количество негативных пометок на кредитнйо истории

In [ ]:
np.mean(df[df['BAD'] == 1]['DELINQ']), np.mean(df[df['BAD'] == 0]['DELINQ'])
#Больше просчрочек

In [ ]:
np.mean(df[df['BAD'] == 1]['CLNO']), np.mean(df[df['BAD'] == 0]['CLNO'])
#Колчиество кредитов практически идентично

In [ ]:
np.mean(df[df['BAD'] == 1]['DEBTINC']), np.mean(df[df['BAD'] == 0]['DEBTINC'])
#Отношение долгов к расходам больше у неплательщиков

In [ ]:
sns.countplot(x = 'JOB', hue='BAD', data = df)
#Весьма интересное наблюдение: хуже платят представители 1, 3 и 6 профессий.
#Попробуем из этого выделить новый признак, и заново построить модель

In [ ]:
plt.hist(df[df['BAD'] == 0]['DEBTINC'])

In [ ]:
plt.hist(df[df['BAD'] == 1]['DEBTINC'])

In [ ]:
X_full = pd.concat([X_train, X_test])
X_full['notgood_job'] = df['JOB'].map(lambda x: 1 if ((x == 1) or (x == 3) or (x == 6)) else 0)

In [ ]:
X_full.head()

In [ ]:
xgb = XGBClassifier(max_depth=6, n_estimators=750, n_jobs=-1, random_state=17)
scaler = StandardScaler()
X_train = X_full.iloc[:int(idx_split), :]
X_test = X_full.iloc[int(idx_split) :, :]
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
xgb.fit(X_train_scaled, y_train)
print("Результат выбранной модели: %s" % roc_auc_score(y_test, xgb.predict_proba(X_test_scaled)[:,1]))

Совсем незначительные улучшения, попробуем ещё.

In [ ]:
#А теперь попробем добавить признак notgood_debtinc
X_full = pd.concat([X_train, X_test])

X_full['notgood_debtinc'] = df['DEBTINC'].map(lambda x: 1 if (((x > 45) and (x <= 100)) or 
                                                                 ((x > 140) and (x < 170))) else 0)

In [ ]:
X_full.head()

In [ ]:
xgb = XGBClassifier(max_depth=6, n_estimators=750, n_jobs=-1, random_state=17)
scaler = StandardScaler()
X_train = X_full.iloc[:int(idx_split), :]
X_test = X_full.iloc[int(idx_split):, :]
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
xgb.fit(X_train_scaled, y_train)
print("Результат выбранной модели: %s" % roc_auc_score(y_test, xgb.predict_proba(X_test_scaled)[:,1]))

In [ ]:
#Никаких улучшений с этим признаком, удаляем. И заново обучим модель.
X_train.drop('notgood_debtinc', axis = 1, inplace=True)
X_test.drop('notgood_debtinc', axis = 1, inplace=True)
xgb = XGBClassifier(max_depth=6, n_estimators=750, n_jobs=-1, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
xgb.fit(X_train_scaled, y_train)

Построим кривые валидации и обучения.

In [ ]:
#Кривые валидации в зависимости от глубины 750 деревьев
train_scores, test_scores = validation_curve(XGBClassifier(n_estimators=750), X_train, y_train,'max_depth', range(1,8), cv = cv, scoring='roc_auc')
plt.plot(range(1,8), np.mean(train_scores, axis = 1), label = 'train set')
plt.plot(range(1,8), np.mean(test_scores, axis = 1), label = 'test set')
plt.legend()
plt.show()

In [ ]:
#Кривые обучения в зависимости от величины выборки
train_sizes, train_scores, test_scores = learning_curve(xgb, X_train, y_train, train_sizes = np.linspace(0.1, 1, 7), cv = cv, scoring = 'roc_auc')
plt.plot(train_sizes, np.mean(train_scores, axis = 1), label = 'train set')
plt.plot(train_sizes, np.mean(test_scores, axis = 1), label = 'test set')
plt.legend()
plt.show()

По обоим графикам можно свидетельствовать о нехватке данных. По validation curve заметно, что схоимость графиков может ещё наблюдаться с более сильным увеличением глубины деревьев. По learning curve результат модели на тренировочных данных быстро выходит на плато, а на тестовых заметен сильный спад и небольшой рост под конец. Безусловно, необходим больший размер выборки.

In [ ]:
result = roc_auc_score(y_test, xgb.predict_proba(X_test_scaled)[:, 1])
print('Результат модели %s' % result)

В принципе, этот результат мы уже видели, когда создали для себя условный бейзлайн при генерации фичей. Если сравнивать его с резульаттом кросс валидации, то потеря качества не особо существенна, менее 1%.

## Выводы.

Не представляю, как раньше банки составляли скорринговые карты вручную, каким образом исследовали зависимости, определяли закономерности. Скорее всего,  основной инструмент, которым приходилось руководствоваться - это личные педубеждения. И, либо на службу нанимались действительно незаурядные личности, обладающие особенным чутьём, либо уже тогда банки уверенно полагались на коллекторов=). В любом случае, в современных условиях необходимо анализировать риски, чтобы корректно выстраивать сратегию развития и дальнейшего существования компании. Без сбора статистической информации это сделать невозможно, а без машинного обучения, её невозможно адекватно оценивать.
Применительно к построенной модели. Не смотря на то, что по кривым валидации и обучения был получен вывод о недостаточной величине обучаемой выборки, а так же о дальнейших возможностях усложнения выбранной модели, результат roc auc в более чем 90% на отложенной выбокре говорит о достаточно хорошем качестве уже сейчас. В качестве улучшения я бы порекомендовал собирать дополнительные признаки, вроде возраста заёмщика, семейного положения, количества детей в семье (вполне возможно, что семьи с детьми тщательнее планируют свой бюджет и в среднем имеют меньшее количество просрочек).